# MELODIES-MONET dev for CAM-chem

This notebook requires approximately 10 GB of memory.

First we need to import the driver.

In [1]:
import sys
sys.path.append('../../')
#print(sys.path)

In [2]:
from melodies_monet import driver

Please install s3fs if retrieving from the Amazon S3 Servers.  Otherwise continue with local data
Please install h5py to open files from the Amazon S3 servers.
Please install h5netcdf to open files from the Amazon S3 servers.


In [6]:
# needed if make changes to code:
%load_ext autoreload

%autoreload 2

### Initiate the analysis class

Now lets create an instance of the python driver analysis class. It consists of 4 main parts; model instances, observation instances, a paired instance of both. This will allow us to move things around the plotting function for spatial and overlays and more complex plots.

In [3]:
an = driver.analysis()

### Control File

Read in the required yaml control file that sets up all the definitons of what we want to pair and plot.

In [4]:
an.control = '../../melodies_monet/examples/yaml/control_cam_chem_example.yaml'
an.read_control()
an.control_dict

{'analysis': {'start_time': '2019-09-01-00:00:00',
  'end_time': '2019-09-09-00:00:00',
  'output_dir': '/glade/work/buchholz/melodies-monet/output',
  'download_maps': False,
  'debug': False},
 'model': {'cam-chem': {'files': '/glade/work/buchholz/CAM_chem_output/melodies_input/CAM_chem_merra2_FCSD_1deg_QFED_world_201909-01-09_small_sfc.nc',
   'mod_type': 'cesm_fv',
   'radius_of_influence': 18000,
   'mapping': {'airnow': {'O3': 'OZONE'}},
   'projection': 'None',
   'plot_kwargs': {'color': 'dodgerblue', 'marker': '+', 'linestyle': '-.'}}},
 'obs': {'airnow': {'use_airnow': True,
   'filename': '/glade/work/buchholz/melodies-monet/melodies_monet/data/AIRNOW_20190901_20190930.nc',
   'obs_type': 'pt_sfc',
   'variables': {'PM2.5': {'unit_scale': 1,
     'unit_scale_method': '*',
     'nan_value': -1.0,
     'ylabel_plot': 'PM2.5 (ug/m3)',
     'ty_scale': 2.0,
     'vmin_plot': 0.0,
     'vmax_plot': 22.0,
     'vdiff_plot': 15.0,
     'nlevels_plot': 23},
    'OZONE': {'unit_scale

### Loading the model data 

The driver will automatically loop through the "models" found in the model section of the control file and create model classes for each. Classes include the label, mapping information, and xarray object as well as the filenames.  Note it can open multiple files easily by including wildcards. Here we are only opening one CAM-chem file.

In [8]:
#%autoreload 2

In [67]:
# testing the new read code
#from new_monetio import read_cesm_fv #Eventually add to monetio itself.
#test = read_cesm_fv.open_mfdataset('/glade/work/buchholz/CAM_chem_output/melodies_input/CAM_chem_merra2_FCSD_1deg_QFED_world_201909.nc')
#test

In [5]:
an.open_models()

cam-chem
{'files': '/glade/work/buchholz/CAM_chem_output/melodies_input/CAM_chem_merra2_FCSD_1deg_QFED_world_201909-01-09_small_sfc.nc', 'mod_type': 'cesm_fv', 'radius_of_influence': 18000, 'mapping': {'airnow': {'O3': 'OZONE'}}, 'projection': 'None', 'plot_kwargs': {'color': 'dodgerblue', 'marker': '+', 'linestyle': '-.'}}
/glade/work/buchholz/CAM_chem_output/melodies_input/CAM_chem_merra2_FCSD_1deg_QFED_world_201909-01-09_small_sfc.nc
**** Reading CESM model output...


In [6]:
an.models

{'cam-chem': <melodies_monet.driver.model at 0x2b6c3a9958b0>}

In [7]:
an.models['cam-chem'].obj

<xarray.Dataset>
Dimensions:  (time: 36, z: 1, lat: 192, lon: 288)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * z        (z) float64 992.5
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 2019-09-01T06:00:00 ... 2019-09-10
Data variables:
    O3       (time, z, lat, lon) float32 dask.array<chunksize=(36, 1, 192, 288), meta=np.ndarray>
    PM25     (time, z, lat, lon) float32 dask.array<chunksize=(36, 1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              fmerra.2.1003.FCSD.f09.qfedcmip.56L.001.branch02
    logname:           buchholz
    host:              cheyenne3
    initial_file:      /glade/p/cesmdata/cseg/inputdata/atm/cam/inic/fv/f.e20...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/met/MERRA2/0....
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_6
    history:           Mon Feb 28 16:25:23 2022: ncks -7 -L 1 --baa=4 --ppc d...
    NCO:               netCDF Operators version 5.0.6 (Homepage = http://nco....

In [8]:
#All the info in the model class can be called here.
print(an.models['cam-chem'].label)
print(an.models['cam-chem'].mapping)

cam-chem
{'airnow': {'O3': 'OZONE'}}


In [9]:
#All the info in the analysis class can also be called.
print(an.start_time)
print(an.end_time)
print(an.download_maps)

2019-09-01 00:00:00
2019-09-09 00:00:00
True


### Open Obs

Now for monet-analysis we will open preprocessed data in either netcdf icartt or some other format.  We will not be retrieving data like monetio does for some observations (ie aeronet, airnow, etc....).  Instead we will provide utitilies to do this so that users can add more data easily.

Like models we list all obs objects in the yaml file and it will loop through and create driver.observation instances that include the model type, file, objects (i.e. data object) and label  

In [10]:
#an.control_dict['obs']

In [11]:
an.open_obs()

In [13]:
#All the info in the observation class can also be called.
#an.obs['airnow'].obj

In [12]:
#This just pairs the data
an.pair_data()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s


In [78]:
#%autoreload 2

In [13]:
#And this generates all the plots.
an.plotting()

Reference std: 15.44077321453672
Reference std: 9.392777428526704
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
